# Tactics

>This document is ©2015-2019 [tom-r](https://github.com/tom-r) and reproduced here from the [original source](https://github.com/tom-r/tactics_pi/blob/e97aede98dc203e6ec1f1ce17c6ff19947fe1b09/tactics_pi.odt), together with the source code containing tactic_pi algorithms and methods under the GNU General Public License v3 (GPL v3) – Likewise, modifications in [DashT software](https://github.com/canne/dashboard_tactics_pi) into those algorithms and their description in this document are ©2019-2020 [canne](https://github.com/canne/dashboard_tactics_pi), and published  under the same license. Please feel free to fork and copy this work  but if you plan to publish it, please keep this notice unmodified in the document together with any copyright statement in the source code.

## Introduction

Everything started with the question : _“Do we make it around the edge of that island when we tack now and sail the same apparent wind angle on the other tack?“_

>**Disclaimer**: This is still alpha code (not even beta), and you should not use it for live – real sailing. I will not be liable for any harm, damage or whatever strange things happen if you use this plugin and rely on its data.

## What Tactics can do?

- Calculate true wind data : TWA, TWD, TWS from true heading (HDT), speed through water (STW) and app. Wind speed (AWS), with optional correction by heel-angle. Calculation is enabled via a preference setting and disables available true wind data from the bus throughout the tactics_pi plugin ;

- Calculate the “leeway”, the boat drift based on heel. A common formula is used for that purpose ;

- Calculate the surface sea current and display it as single instruments (current speed/direction) as part of the “Bearing compass” or as overlay on the chart (semi transparent). The routines take boat heel and leeway into account. If you don't have a heel sensor, there is a simply workaround, see below. Current display on the chart can be disabled by a preference setting ;

- Calculate and display the boat laylines for the current tack, and the same TWA on the other tack. Sea current is taken into account, if available ! Laylines may be toggled on/off.  Adjustable length and max. width (triangle, with one corner at the boat) of the boat laylines. The layline width reflects the boat's yawing (COG changes over time) ;

- You can load a polar file and calculate/display performance data, like actual VMG (velocity made good up-/downwind), Target-VMG, Target-TWA  (the opt. TWA up-/downwind), CMG (course made good towards a waypoint), Target-CMG (opt. CMG angle and speed), polar speed (the speed you should be able to sail at current TWA/TWS based on your polar) ;

- It can display the actual polar as overlay on the chart, including markers for Target-VMG/CMG angles. Just adopt your course and place your heading pointer on one of the markers, and you sail Target-VMG/CMG based on your polar ;

- You can set a (one !) temporary tactics waypoint and display the laylines to the mark, based on a Target-TWA calculation, while taking your polar into account ;

- It has a “dial instruments” called “Bearing compass”. Boat true heading (HDT)  points “up”, it shows the boat laylines as well, the surface current, a pointer to the waypoint (either set manually as the temporary Tactics waypoint or read from a NMEA RMB sentence), needles for AWA and TWA, markers for the Target-VMG/CMG angles ;

- It has a “Polar compass” instrument, graphically displaying the actual polar ring and markers for Bearing to WP, Target-VMG angles and Target-CMG angles ;

- It has an “Average Wind” instrument with an adjustable averaging time, which displays graphically the average wind and its deviations to port / starboard

- It can create specific NMEA performance records with the purpose to export them to the displays of your instruments. You can now e.g. calculate the polar target speed in the plugin and send it to your instrument display outside in the cockpit. Currently only available for NKE, but may be enhanced in the future ;

- It can export the data from BaroHistory, WindHistory and PolarPerfomance instruments to a flat text file with selectable export rate and column separator (via ini file). The export will either create a new, or append to an existing export file ;

- Various Tactics settings are grouped in a separate tab in the Preferences configuration dialog of _DashT_. To access the preferences dialog, right mouse click on any _DashT_ instrument, then select “Preferences" (**8**);

<img src="img/s_001_DashT_prefs_context_menu.png"
alt="Context menu" width="200">[(zoom)](img/001_DashT_prefs_context_menu.png)

- You'll find all seetings in a separate tab _“Tactics Performance Parameters”_ (**1**):

<img src="img/s_005_DashT_Tactics_Perf_Parameters_dlg.png"
alt="Tactics Performance Parameter Access" width="200">[(zoom)](img/005_DashT_Tactics_Perf_Parameters_dlg.png)

- There are pop-up _ToolTips_ available for almost all of the Tactics preferences. Just hover the mouse over any of the settings to get explanation and usage description.

<img src="img/s_005_DashT_Tactics_Parameters_Tooltip.png"
alt="Tooltips" width="400">[(zoom)](img/005_DashT_Tactics_Parameters_Tooltip.png)

Below is a screenshot depicts all the Tactics parameters which can be changed from the Preferences dialog. There are some additional ones, explained later on in this document which can be changed in ini/conf-file.

<img src="img/s_010_DashT_Tactics_All_Perf_Params_In_Dialog.png"
alt="All parameters" width="400">[(zoom)](img/010_DashT_Tactics_All_Perf_Params_In_Dialog.png)

## Prerequisites

- You will need to activate OpenGL, if you want to use the chart overlay functions ;
- SOG, COG from the GPS ;
- True Heading from an electronic compass. If not available, magnetic heading will do, as long as you have magnetic variance available, either from the GPS RMC sentence or from the wmm_pi plug-in - note that the wmm_pi plug-in's value has the priority (Dashboard heritage) so you may want to disable wmm_pi plug-in if your own preference is the RMC sentence ;
- Boat speed through water **STW** from a log / “paddlewheel” sensor (keep it clean!) ;
- Apparent wind angle **AWA** and Apparent Wind Speed **AWS** ;
- Heel sensor which supplies your boat's heel angle, also calld "roll". If not available, there's a workaround with manual input in preferences ;
- You need a polar file of your boat to use all polar based performance calculations, explained in a separate section in this document ;
- Calibrated sensors or corection to AWA, Compass HDG/HDT, STW (Speed through water), and AWS (apparent wind speed), as good as possible. Especially the compass heading calibration tends to be neglected. However, this is vital for a  proper surface current calculation. All I can say is :   sh*** in – sh*** out …   ;-) .

The terms CMG and VMG used in this documentation are not unique across instrument manufacturers and other sailing software packages, even in OpenCPN and in Dashboard "VMG" is used in routing functions! This software will still use both terms, familiar to the intended target users, both in the software and here, in this documentation. However you can change both terms by setting 2 preferences in the ini/conf-file file. See the corresponding chapter explaing the parameters in that file.

See also the Terminology at the very end for explanation of terms

## Without Polar

The instruments and functions in this chapter do not require that you have your boat's polar.

### Calculate true wind data

The plug-in can calculate true wind data (TWA, TWS, TWD) but keeps the calculated data inside the plug-in. (it does not broadcast it to OpenCPN!)

True wind calculation is done silently if you don’t have TWA, TWS, TWD available in your NMEA stream.  Furthermore you can force the true wind calculation in the plug-in by a preference setting. If the tick _“Force True Wind Calculation”_ is set 

<img src="img/s_015_DashT_Tactics_tw_calcs_tickboxes.png"
alt="DashT Tactics TW Tickboxes" width="300">[(zoom)](img/015_DashT_Tactics_tw_calcs_tickboxes.png)

It does not matter if TWA, TWS and TWD are already available on the system or not. Calculation is done in the plug-in then.  It does calculate TWA, TWS and TWD.

This is useful in case, for example if you have a heel sensor but which is not integrated in your instrument bus. You can use the corrections to get more accurate true wind data by taking into account the heel (roll).

Input is AWA, AWS, STW, and for TWD also true heading HDT.

If you don't have HDT on your system bus (but only HDG), you can use wmm_pi plug-in: it supplies the magnetic variation and if it is enabled, it value is fed into _DashT_ by OpenCPN and its value is taken into account to calculate HDT from HDG.

If you have a heel sensor in your system, and its data is available to _DashT_, you can use the three following corrections:

1. **Correct STW with Leeway** : the plug-in can calculate your leeway (drift angle) based on on your heel sensor (see below). That means your boat is possibly moving sideways, which adds an error to the True Wind calculation. Standard instruments normally do not take this effect into account, as far as I know. NKE does this correction in its regatta processor only, but not on their normal instruments.
1. **Correct AWS/AWA with Heel** : This option corrects your AWS and AWA data by the heel angle. Use this option with great care !. Manufacturers normally already do correct this, if you have a heel sensor integrated in your instrument bus. O will simply receive the already corrected data for AWS / AWA then. The result would be wrong data ! I implemented this option for those sailors using an external (or DIY) heel sensor, which is not recognized by their instrument system. You'll get a warning popup as soon as you set the tick.
1. **Use SOG instead of STW for True Wind Calc** : Replaces STW (Speed through water, the “log”) with SOG (from the GPS) in the internal true wind calculation.  The idea is simply to have a fallback for the true wind calculation in case your log fails and also to eliminate side effects on the calculation by surface current. Some like the option to refer to the ground and some prefer to refer to the water - make your choice!

>**NOTE**: if you have issues with the true wind calculation in _DashT_, take a look at the OpenCPN log-file. _DashT_ collects the information needed by the Tactics algorithm and if something is missing, it is reported in the log file to help the troubleshooting.

### Calculate Leeway

Leeway describes the drift of the boat due to the force of the wind. Leeway is the basic input for the surface current calulation described later on. Input for the leeway calculation is your heel angle. Normally you'd say : the more you heel, the more you drift. But that's only part of the truth. Other significant inputs are boat speed and the shape of your hull.

A widely (NKE, B&G,...) used formula calculates the leeway with 3 input values : heel, boat speed (STW), and hullshape-factor.

The attribute in the preferences is called _“Boat's Leeway factor [0..20]:”_

The input range is 0...20, 10 is a good value to start with.

f you don't have a heel sensor on board, you can either set a fixed value ( e.g. 0 when motoring without sails), or try to set up a very simple _“heel polar”_:

<img src="img/s_020_DashT_Tactics_heel_polar_preferences.png"
alt="Heel Polar" width="300">[(zoom)](img/020_DashT_Tactics_heel_polar_preferences.png)

The idea of the _“heel polar”_ is that almost every magnetic compass installed in the cockpits has a scale, where you can read the degrees of heel: simply compare the _DashT_ Dashboard instruments display, for TWA and TWS, read the values from the scale and put it into the small table above.

>**NOTE**: Please make sure to read the True Wind Angle (TWA) and the True Wind Speed (TWS) and not apparent wind angle and speed (AWA/AWS).

I tried it on my own boat, comparing the heel polar values with those of my sensor. It works astonishingly well.

Even if you use the heel-polar, you have to estimate the _“Boat's Leeway factor [0..20]:”_

You have 3 choices for heel input, depending on where you set the radio button in the preferences. You can switch the radio buttons forth and back while sailing to compare the results, or as a tactician to take into account different sailing conditions.

<img src="img/s_025_DashT_Tactics_Leeway_factor_preferences.png"
alt="Leeway factor" width="300">[(zoom)](img/025_DashT_Tactics_Leeway_factor_preferences.png)

The attribute _“fixed/max Leeway [°]:”_ has a dual purpose:

1. The given value is always taken into account as maximum possible Leeway value. In the screenshot below, I set it to 30°. If your heel polar or calculation with the formula above outputs values >30°, the program takes 30° ;
1. If you set the radio button here, the routines always take 30°, no matter what your sensor calculates or your heel-polar would tell you.

### Calculate the surface current

If you compare your HDT and COG vectors in OpenCPN (the 2 forward vectors on the chart at your boat), the difference between both is a mixture between Leeway (the boat's drift) and surface current. Once we can determine Leeway, the rest is surface current.

The surface current calculation is simply a triangle calculation with vectors.

Always seen from the current position, the first vector is HDT (degrees) / STW (length).

As your boat drifts with the wind, the second vector is “course through water” (CRS, degrees) and STW (length).

Course through water is actually HDT with applied leeway.

The resulting vector between CRS/STW and COG/SOG is the surface current.

<img src="img/s_030_DashT_Tactics_current_vector_calculation.png"
alt="Current vector calculation" width="240">[(zoom)](img/030_DashT_Tactics_current_vector_calculation.png)

To calculate the current, you need as input HDT, STW, Leeway, COG and SOG and your GPS latitude / longitude.

In the preferences you can set the damping factor for the current calculations : the lower the values are the more filtering is applied, and the reading gets more stable. On the other hand, it starts lagging a bit.
The lower the value, the more damping is applied. You can do experimenting in the range of 0.001  to 0.025. Keep this value at the lower end, then start to increase, until it gets unstable.

<img src="img/s_035_DashT_Tactics_current_damping_factor_preferences.png"
alt="Current Damping Factor" width="300">[(zoom)](img/035_DashT_Tactics_current_damping_factor_preferences.png)

>**NOTE**: Tactics algorithms currently always corrects STW with Leeway for the current calculation, independent of the preference setting  _“Correct STW with leeway”_. This may change in a future release but even then the default setting will remain as it is.

### Current arrow on chart

You can show a semi transparent blue current symbol underneath your boat, showing the surface current direction.

<img src="img/s_040_DashT_Tactics_Current_Arrow.png"
alt="Current Arrow" width="150">[.](img/040_DashT_Tactics_Current_Arrow.png)

To activate the current display on the chart by default, upon program start, navigate to the Preferences dialogue and set the tick _“Display Current on Chart (OpenGL)”_. The preference is only setting the default; while navigating in OpenCPN, you can turn the current overlay display on / off as you like:

Just right-click on any _DashT_ instrument and select _“Show Current”_ in the

##### contextual menu

<img name="contextualmenu" src="img/s_045_DashT_Tactics_Current_Arrow_Toggle.png"
alt="Toggle Current" width="150">[.](img/045_DashT_Tactics_Current_Arrow_Toggle.png)

This will toggle the current display arrow on/off.

### Boat laylines

You can show the boat laylines on the chart. They refer to COG.

<img src="img/s_050_DashT_Tactics_Laylines.png"
alt="Laylines" width="200">[(zoom)](img/050_DashT_Tactics_Laylines.png)

The colours mean:

- _red_ = wind from port ;
- _green_ = wind from starboard.

The width is defined by the yawing of your boat, the more there is yaw, the wider the laylines do get.

The second layline (green in this example) simply shows you where you would get on the other tack / gybe when you sail the same TWA after the tack. Leeway and current are taken into account for the calculation of the second layline.

In the Preferences, you can set the following options:

<img src="img/s_055_DashT_Tactics_Laylines_Options.png"
alt="Layline options" width="300">[(zoom)](img/055_DashT_Tactics_Laylines_Options.png)

The _“Layline width damping factor”_ is the rate how fast the layline width reacts on COG changes.

It's done with exponential smoothing, the smaller the factor, the higher the damping rate.

You can define the length of the laylines on the chart, as well as a minimum and maximum width.

If you don't like the yawing effect simply set min and max both to 1 or 2 degrees.

To toggle the layline display on the chart on/off, right-click on any _DashT_ instrument and select or unselect _“Show laylines”_ from the [contextual menu](#contextual-menu).

### Wind barbs

You can also show a wind barb at the boat position, showing you direction and speed (feather length) in 5 kt steps.

<img src="img/s_060_DashT_Tactics_Windbarb.png"
alt="Windbarb" width="150">[(zoom)](img/060_DashT_Tactics_Windbarb.png)

To activate the wind barb display on the chart by default, upon program start, navigate to the Preferences dialogue and set the tick _“Display Wind Barb on Chart (OpenGL)”_.

The preference is only setting the default; while navigating in OpenCPN, you can turn the windbarb overlay display on / off as you like: to toggle the windbarb display on the chart on/off, right-click on any _DashT_ instrument and select or unselect _“Show Windbarb”_ from the [contextual menu](#contextual-menu).

Tactics brings in some improvements to the windbarb presentation, inherited from other OpenCPN plug-ins, it shows the wind barbs up to 47 knots(!) correctly with the following mapping:

<img src="img/s_065_DashT_Tactics_Windbarb_scale.png"
alt="Windbarb scale" width="300">[(zoom)](img/065_DashT_Tactics_Windbarb_scale.png)

### Current Direction and Speed

>**NOTE** In _DashT_ the Tactics instruments are identified by an up-arrow **↑** - for performance boost they provide!

<img src="img/s_070_DashT_Tactics_CurDir_CurSpeed.png"
alt="Current Direction and Current Speed" width="150">[(zoom)](img/070_DashT_Tactics_CurDir_CurSpeed.png)

Current direction means: “current sets into”. 

### Leeway

<img src="img/s_075_DashT_Tactics_Leeway.png"
alt="Leeway" width="150">[(zoom)](img/075_DashT_Tactics_Leeway.png)

Boat drifts 0.5° to starboard due to [heel](../dashinst/dashinst.html#Heel).

### Average Wind Instrument

The graphical instrument shows the average wind and its deviations to both sides.

This instrument is also used to control and display the result of a background processing average wind calculation using Tactics double exponential smoothing algorithm. It is recommended to have it in your instrument catalog if you plan to use any other instrument requiring the information about the average wind.

<img src="img/s_090_DashT_Tactics_Avg_wind_instrument_short_and_long.png"
alt="Average Wind Insrtrument" width="300">[(zoom)](img/090_DashT_Tactics_Avg_wind_instrument_short_and_long.png)

The curve is centered on the average wind, green means the wind is on starboard of the actual average wind, red means it blows more from port.

One can adjust the averaging time in steps of 1 [min] between 4 and 30 mins.

“Short” integration time is from 10 .. 50 % of that time, default is 25 % integration time. That value has its own slider which is depicted with a red or green thick bar and the corresponding, sliding minimum and maximum value lighter bars in the upper part of the diagram. This feature can be turned off in the ini/conf-file, see the corresponding section explaining its settings.

The red number in the center is the average wind direction, left and right is the min and max (unfiltered) wind angles to either side.

The very thin lines are the unfiltered wind direction input from the instruments.

To adjust the time average, just pull the slider left / right.

The vertical scale is [minutes], short dashes every minute, full horizontal line every 5 minutes.

The instrument has its own timer, so it’s independent of the data connection speed or rate.

The idea of this instrument is following: if you sail in puffy, changing winds, that you see graphically when the wind changes to the other side. In theory, you should tack, as soon as the wind veers away and crosses the average wind direction 

>**NOTE**: _Race DashT Mark_ is another instrument which can be used to determine if a tack imposes or not. It provides your location on the ladder rungs, giving also the idea is the wind shift a lifter or a header. Please read more from the corresponding section.

### TWA to Waypoint

To use this instrument, you need to drop a Tactics waypoint on the chart, using the right-click context menu on the chart canvas. Altenatively, you can place a waypoint in GPS so that it appears to OpenCPN in an RMB-sentence. If you define both, Tactics WP takes the preference.

This instrument requires also the averaged wind data processing running continuously in the background. How to obtain it will be explained below.

<img src="img/s_080_DashT_Tactics_twa_to_wp.png"
alt="TWA to Waypoint" width="150">[(zoom)](img/080_DashT_Tactics_twa_to_wp.png)

This instrument is meant to check  the TWA on the (new) tack prior to sailing the tack/gybe manoeuvrer.

Once you know the TWA angles of your sails, this is very useful e.g. to see which of your downwind sails to use on the next tack.

The instrument shows three data values:

1. minimum instantenous or short term average TWA
2. long term average TWA
3. maximum instantenous or short term average TWA

All 3 values are exactly the same data as used in the Average Wind Direction instrument, just recalculated to TWA towards a waypoint.

>**NOTE 1**: one cannot refer from <min TWA > and <max TWA> to  port / starboard side, values are simply sorted in the display the low and high.

>**NOTE 2**: This instrument depends on the Average Wind Direction instrument which has been extented in _DashT_ to include two range of averaging time, “normal” and “short” time span. This particular instrument using that data is set, in _DashT_ to use the “short” time range (like for regatta) by default. Since this feature can be turned off in the Average Wind Direction instrument, this instrument automatically returns also back to “normal”, i.e to long average wind integration and smoothing time like in the Tactics v1.0.11. It is also possible to select the long integration time permanently for this instrument only, in the ini/conf-file, see the corresponding section.

Once the wind veers from port to starboard side on full downwind courses the display can look like this: 

<img src="img/s_085_DashT_Tactics_twa_to_wp_downwind.png"
alt="TWA to WP downwind example" width="150">[(zoom)](img/085_DashT_Tactics_twa_to_wp_downwind.png)

That means, average TWA is 178° and the wind deviates to 154° to one tack and 160° to the other tack.

Default time is 4 minutes, maximum 30 minutes. _“Short”_ integration time can be adjusted (in ini/conf-file) from 10 .. 50% of that time, default is 25% of the overall integration time.

<img src="img/s_090_DashT_Tactics_Avg_wind_instrument_short_and_long.png"
alt="Average Wind Insrtrument" width="150">[(zoom)](img/090_DashT_Tactics_Avg_wind_instrument_short_and_long.png)

To use Average Wind, both _“Short”_ and _“Long”_ in all _DashT_ instruments needing that information, just define the ‘Average Wind Direction’ instrument in your list of instruments, open it once (per session) and move the slider to the integration time of your preference. You can close the instrument afterwards, just keep it in the list of instruments.

More use cases how to use this instrument to select which gennaker/spi or sail for the next tack:

Let’s say we have 2 gennakers on board, the first one start to work at 105°TWA, has its best perfomance at 120-125° and drops out at 135° TWA.

The second gennaker starts at 115°, has its best performance at 130-140° TWA and can be used all down to 160° TWA.

With the example values (113 – 129 – 151) from the first screenshot above you cannot stay on the direct course line to the WP with the second gennaker all time as it starts at 115° (whereas we recorded 113° TWA...),  but overall it fits with its optimum performance (130-140°) much better to the average wind direction (129° TWA) than the first gennaker.

>**NOTE**: _Race DashT Mark_ is another instrument which can be used for this type of optimization process, in case the race route can be marked in advance, in which case it calculates the TWA values in clear for two legs ahead. Please see the corresponding section for more details.

## With Polar

The instruments and functions in this chapter require your boat's polar.

You can load a polar file by defining one in the Preferences dialog:

<img src="img/s_095_DashT_Tactics_Load_Polar_File.png"
alt="Load Polar File" width="300">[(zoom)](img/095_DashT_Tactics_Load_Polar_File.png)

Click on the load button and select a polar file.

The format is the same that the _polar_pi_ plug-in uses. Use that tool to display the polar, there is no polar visualization in _DashT_.

>**NOTE**: If you do not have a polar, the Weather Routing plug-in of OpenCPN comes with quite a few polars of popular boats. Maybe you can find a polar close enough of your boat in there.

For racers, it is a good idea to prepare a few alternative polars simply by reducing the the polar speed by, say 10 percent: they can be used in bad weather conditions or by a single hand sailor who is not able to push the boat continuously to its limits.

When Tactics is loading the polar file, it is wirting it into a 2 dimensional static array with 181 lines for each TWA degree (0...180°) and 61 rows (0...60) for each knot of windspeed. It's a simple lookup table for the latter use, with the purpose to reduce the processor load and increase access speed.

Polar Loading Procedure :

1. The TWA = 0 row and the TWS = 0 column is filled with zeros
1. The rest of the array is filled with NAN values. 
1. The values from the polar file are placed at their corresponding spots in the array
1. The missing data in between given values (= not NAN) is then filled with average values.

>**NOTE**: only polars with TWA / TWS / STW, where TWS and STW are in knots do make sense 

While there is averaging to fill in the missing values, there is no extrapolating beyond the last valid values.

If you run in a 30kt wind, and your polar ends at 25 kts, then the performance instruments will give you a “no polar data” text. Please consider at that moment (if you have time) to turn on polar_pi / vdr_pi plug-ins to record new data since you obviously are a serious racer!

The only exception of the extrapolation is the range between the 0 kts windspeed and the first given value, where a simple averaging between the two is done.

<img src="img/s_100_DashT_Tactics_Polar_extrapolation.png"
alt="Polar Extrapolation" width="200">[(zoom)](img/100_DashT_Tactics_Polar_extrapolation.png)

It is possible to extract the generated, extrapolated polar file by providing an export path in a variable in ini/conf-file. Please see the corresponding chapter explaining each variable.

<img src="img/s_105_DashT_Tactics_Polar_input_example.png"
alt="Polar input example" width="600">[(zoom)](img/105_DashT_Tactics_Polar_input_example.png)

And this is the corresponding lookup table dumped to a file using the ini/conf-file key _“PolarLookupTableOutputFile”_

<img src="img/s_110_DashT_Tactics_Polar_output_example.png"
alt="Polar output example" width="600">[(zoom)](img/110_DashT_Tactics_Polar_output_example.png)

>**NOTE**: Only True Wind Speed (TWS) values are averaged. True wind angle is still rounded to the next full knot. For testing, Tactics has implemented full averaging of both TWS and TWA as well, but live comparisons did not show improvement. Therefore this method of averaging TWS only has been retained. TWA  will not have better accuracy than plus or minus one degree, due to upwash issues, position and accuracy of the wind vane, compass accuracy, etc. 

### Display polar on chart

<img src="img/s_115_DashT_Tactics_Polar_on_chart.png"
alt="Polar on chart" width="200">[(zoom)](img/115_DashT_Tactics_Polar_on_chart.png)

You can display the polar as overlay on the chart (OpenGL is required). The size of the different polar “rings” is normalized, they all have the same size. Nevertheless, the plug-in always shows the current / correct polar ring. The polar orientation is related to the true wind direction, and it shows blue markers for the Target-VMG angles up- and downwind, and red markers for the Target-CMG angles (if you have an active NMEA-RMB-sentence or a Tactics_pi waypoint set).

There’s also a small blue HDT line diplayed from the boat. The additional marker line for HDT is only to help you to distinguish which of the two OpenCPN red, default marker lines is for heading and which one is for the course over ground.

To sail Target-VMG / Target-CMG angle, simply steer the blue HDT pointer on one of the Target-VMG / - CMG markers.

As illustrated in the above screenshot, there can be 2 red CMG markers, the preferred one has a bigger size! 

To activate the polar display on the chart by default, upon program start, navigate to the Preferences dialogue and set the tick “Show polar on chart (OpenGL)”.

<img src="img/s_120_DashT_Tactics_Polar_chart_activation_pref.png"
alt="Activate the polar by default" width="300">[(zoom)](img/120_DashT_Tactics_Polar_chart_activation_pref.png)

The preference is only setting the default; while navigating in OpenCPN, you can turn the polar overlay display on / off as you like: to toggle the polar display on the chart on/off, right-click on any _DashT_ instrument and select or unselect _“Show Polar”_ from the [contextual menu](#contextual-menu).

## Performance data

Tactics performance algorithms are used, among the others in following numerical instruments. Let's first take a look the following vector chart as for reference on the different terms in relation to the polar curve:

##### vector graph

<img src="img/s_125_DashT_Tactics_Polar_rel_vectors.png"
alt="XXXX" width="400">[(zoom)](img/125_DashT_Tactics_Polar_rel_vectors.png)

>**NOTE**: We need to presume that you are familiar with the above terminology - if not, read about [VMG](https://sailzing.com/velocity-made-good-vmg) to start with - but if you do not agree, say with the [CMG definition](https://en.wikipedia.org/wiki/Velocity_made_good#Confusion_with_VMC) you are free to change the abbreviations used in the application using the ini/conf-file settings, see the corresponding chapter in _DashT_ documentation. All requests to change the nomenclature will be rejected since one has this option. The vector algebra does not move.

Some of the instruments are not showing a single value like on Dashboard, buth the value is split in two, displaying first a percentage value and then the value from which the percentaget is taken. Let's take an example of the polar performance, displaying the percentage of your current speed of the polar speed of your boat at that moment:

### Polar speed

<img src="img/s_130_DashT_Tactics_Polar_speed_num.png"
alt="Polar speed" width="150">[(zoom)](img/130_DashT_Tactics_Polar_speed_num.png)

This is simply the reference of what speed we should be able to sail based on our current TWA / TWS values. The % value is the reference to STW. In this example, we are currently doing (only) 51%, of what the polar has stored as optimum value. According to the polar we should be able to do 12.17 knots.

This is _“Act.Boat Speed”_ compared to the _“Polar Speed”_ in the [vector graph](#vector-graph).

Values below 100% means that we are slower than the polar says, above 100% means that we are faster than the polar - if that persists, we should run the _vdr_pi_ plug-in or _DashT_ database streamer to record the new data since obviously our polar is not good. 

This instrument is useful in crosswind / reaching conditions without a waypoint.

It shows the optimum speed for the given wind conditions.

<img src="img/s_.png"
alt="XXXX" width="200">[(zoom)](img/.png)